# Basic SVM for Market Trend Prediction

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Download historical stock data for Apple Inc.
ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2025-01-01")

# Safely select price column ('Adj Close' if available, else 'Close')
price_col = 'Adj Close' if 'Adj Close' in data.columns else 'Close'

# Ensure chosen column exists
if price_col not in data.columns:
    raise ValueError(f"Column '{price_col}' not found. Available columns: {data.columns}")

# Feature Engineering: Compute daily returns, moving average, and volatility
data['Return'] = data[price_col].pct_change()
data['MA20'] = data[price_col].rolling(window=20).mean()
data['Volatility'] = data['Return'].rolling(window=20).std()
data.dropna(inplace=True)

# Define trend (bullish if return positive, bearish otherwise)
data['Trend'] = np.where(data['Return'] > 0, 'bullish', 'bearish')

# Define features and target variable
features = ['MA20', 'Volatility']
X = data[features]
y = data['Trend']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize and train the SVM classifier
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# Predict and evaluate
predictions = svm.predict(X_test)

print("Basic SVM Accuracy:", accuracy_score(y_test, predictions))
print("Classification Report:\n", classification_report(y_test, predictions, zero_division=0))


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Basic SVM Accuracy: 0.5451612903225806
Classification Report:
               precision    recall  f1-score   support

     bearish       0.00      0.00      0.00       141
     bullish       0.55      1.00      0.71       169

    accuracy                           0.55       310
   macro avg       0.27      0.50      0.35       310
weighted avg       0.30      0.55      0.38       310



# Hyperparameter Tuning and Kernel Selection

In [8]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define a simplified parameter grid
param_grid = {
    'kernel': ['linear', 'rbf'],  # Testing only two kernels for speed
    'C': [1, 10],
    'gamma': ['scale']
}

# Initialize GridSearchCV with 3-fold cross-validation for quicker training
grid_search = GridSearchCV(
    estimator=SVC(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='accuracy'
)

# Fit the model
grid_search.fit(X_train, y_train)

# Retrieve the best estimator and display hyperparameters
best_svm = grid_search.best_estimator_
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the tuned model
predictions = best_svm.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Advanced SVM Accuracy:", accuracy)
print("Advanced Classification Report:\n", classification_report(y_test, predictions, zero_division=0))


Best Hyperparameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Advanced SVM Accuracy: 0.5451612903225806
Advanced Classification Report:
               precision    recall  f1-score   support

     bearish       0.00      0.00      0.00       141
     bullish       0.55      1.00      0.71       169

    accuracy                           0.55       310
   macro avg       0.27      0.50      0.35       310
weighted avg       0.30      0.55      0.38       310

